In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('train.csv')
final = pd.read_csv('test.csv')

In [3]:
def does_does_not(cell):
    if cell >= 1:
        return 1
    else:
        return 0
    

def clean1(df):
    df.rename(columns= {col: col.lower().replace(' ','_') for col in df.columns}, inplace=True)
    
    df.drop([
    'alley', 'lot_frontage', 'mas_vnr_type', 'fireplace_qu', 'pool_qc', 'misc_feature',
    'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2',
    'bsmtfin_sf_2', 'bsmt_unf_sf', 'bsmt_full_bath', 'bsmt_half_bath',
    'garage_type', 'garage_yr_blt', 'garage_finish', 'garage_area', 'garage_qual', 'garage_cond',], axis=1, inplace=True)
    
    df['fence'] = df['fence'].notnull().astype('int')
    
    features_replace = ['pool_area' , 'garage_cars', 'fireplaces', 'total_bsmt_sf', 'mas_vnr_area']
    for i in features_replace:
        df[i] = df[i].map(does_does_not)
    
    df.rename(columns={
    'pool_area': 'has_pool',
    'garage_cars': 'has_garage', 
    'fireplaces': 'has_fireplace',
    'total_bsmt_sf': 'has_bsmt',
    'mas_vnr_area': 'has_mas_vnr',
    'fence': 'has_fence'}, inplace=True)
    
    return df.head(15)

In [4]:
clean1(df)

,id,pid,ms_subclass,ms_zoning,lot_area,street,lot_shape,land_contour,utilities,lot_config,...,3ssn_porch,screen_porch,has_pool,has_fence,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
0,109,533352170,60,RL,13517,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2010,WD,Normal,130500
1,544,531379050,60,RL,11492,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,4,2009,WD,Normal,220000
2,153,535304180,20,RL,7922,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,1,2010,WD,Abnorml,109000
3,318,916386060,60,RL,9802,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2010,WD,Normal,174000
4,255,906425045,50,RL,14235,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,138500
5,138,535126040,20,RL,16492,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,6,2010,WD,Normal,190000
6,2827,908186070,180,RM,3675,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2006,New,Partial,140000
7,145,535154050,20,RL,12160,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,1,0,5,2010,COD,Abnorml,142000
8,1942,535353130,20,RL,15783,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,1,400,6,2007,WD,Normal,112500
9,1956,535426130,60,RL,11606,Pave,IR1,HLS,AllPub,Inside,...,0,0,0,0,0,9,2007,WD,Family,135000


In [5]:
# run a quick and simple grid search
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV

In [6]:
df.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_area', 'street',
       'lot_shape', 'land_contour', 'utilities', 'lot_config', 'land_slope',
       'neighborhood', 'condition_1', 'condition_2', 'bldg_type',
       'house_style', 'overall_qual', 'overall_cond', 'year_built',
       'year_remod/add', 'roof_style', 'roof_matl', 'exterior_1st',
       'exterior_2nd', 'has_mas_vnr', 'exter_qual', 'exter_cond', 'foundation',
       'has_bsmt', 'heating', 'heating_qc', 'central_air', 'electrical',
       '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area',
       'full_bath', 'half_bath', 'bedroom_abvgr', 'kitchen_abvgr',
       'kitchen_qual', 'totrms_abvgrd', 'functional', 'has_fireplace',
       'has_garage', 'paved_drive', 'wood_deck_sf', 'open_porch_sf',
       'enclosed_porch', '3ssn_porch', 'screen_porch', 'has_pool', 'has_fence',
       'misc_val', 'mo_sold', 'yr_sold', 'sale_type', 'sale_condition',
       'saleprice'],
      dtype='object')

In [7]:
y = df['saleprice']
features = ['lot_area', 'overall_qual', 'overall_cond',
    'has_mas_vnr', 'has_bsmt', 'has_fireplace', 'has_garage', 'has_pool', 'has_fence'
    ]

X = df[features]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [9]:
lr = LinearRegression()
lr.fit(X_train_sc ,y_train)

print('train', lr.score(X_train_sc, y_train))
print('test', lr.score(X_test_sc, y_test))

train 0.699588817834
test 0.720172543242


In [10]:
enet = ElasticNet()
params = {
    'alpha': np.arange(.01, .03, .005),
    'l1_ratio': np.arange(.01, 1.0, .05)
}
gs = GridSearchCV(enet, params)
gs.fit(X_train_sc, y_train)

print(gs.best_params_)
gs.score(X_test_sc, y_test)

{'alpha': 0.014999999999999999, 'l1_ratio': 0.060000000000000005}


0.71965351927239241

# Final

In [21]:
# clean1(final)

ValueError: labels ['alley' 'lot_frontage' 'mas_vnr_type' 'fireplace_qu' 'pool_qc'
 'misc_feature' 'bsmt_qual' 'bsmt_cond' 'bsmt_exposure' 'bsmtfin_type_1'
 'bsmtfin_sf_1' 'bsmtfin_type_2' 'bsmtfin_sf_2' 'bsmt_unf_sf'
 'bsmt_full_bath' 'bsmt_half_bath' 'garage_type' 'garage_yr_blt'
 'garage_finish' 'garage_area' 'garage_qual' 'garage_cond'] not contained in axis

In [25]:
# X_final = final[features]
# X_final_sc = ss.transform(X_final)
# lr_predictions = lr.predict(X_final_sc)
# predictions = gs.predict(X_final_sc)

In [30]:
# final['SalePrice'] = predictions

# final[['id','SalePrice']].to_csv('./Submits/post_clean_1.csv', index=False)

# Cleaning 2 and EDA 2

In [ ]:
# def house_hist(df=df_nums, rows=18, cols=2):
#     fig, axes = plt.subplots(rows, cols, figsize=(20,100))
#     label = list(df.columns)
#     i = 0
#     for row in range(axes.shape[0]):
#         for col in range(axes.shape[1]):
#             ax = axes[row][col]
#             df[label[i]].hist(ax=ax)
#             ax.set_title(label[i])
#             i += 1
#     plt.tight_layout()

In [59]:
df.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_area', 'street',
       'lot_shape', 'land_contour', 'utilities', 'lot_config', 'land_slope',
       'neighborhood', 'condition_1', 'condition_2', 'bldg_type',
       'house_style', 'overall_qual', 'overall_cond', 'year_built',
       'year_remod/add', 'roof_style', 'roof_matl', 'exterior_1st',
       'exterior_2nd', 'has_mas_vnr', 'exter_qual', 'exter_cond', 'foundation',
       'has_bsmt', 'heating', 'heating_qc', 'central_air', 'electrical',
       '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area',
       'full_bath', 'half_bath', 'bedroom_abvgr', 'kitchen_abvgr',
       'kitchen_qual', 'totrms_abvgrd', 'functional', 'has_fireplace',
       'has_garage', 'paved_drive', 'wood_deck_sf', 'open_porch_sf',
       'enclosed_porch', '3ssn_porch', 'screen_porch', 'has_pool', 'has_fence',
       'misc_val', 'mo_sold', 'yr_sold', 'sale_type', 'sale_condition',
       'saleprice'],
      dtype='object')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 60 columns):
id                 2051 non-null int64
pid                2051 non-null int64
ms_subclass        2051 non-null int64
ms_zoning          2051 non-null object
lot_area           2051 non-null int64
street             2051 non-null object
lot_shape          2051 non-null object
land_contour       2051 non-null object
utilities          2051 non-null object
lot_config         2051 non-null object
land_slope         2051 non-null object
neighborhood       2051 non-null object
condition_1        2051 non-null object
condition_2        2051 non-null object
bldg_type          2051 non-null object
house_style        2051 non-null object
overall_qual       2051 non-null int64
overall_cond       2051 non-null int64
year_built         2051 non-null int64
year_remod/add     2051 non-null int64
roof_style         2051 non-null object
roof_matl          2051 non-null object
exterior_1st       20

In [14]:
df.corr()['saleprice'].sort_values(ascending=False)

saleprice          1.000000
overall_qual       0.800207
gr_liv_area        0.697038
1st_flr_sf         0.618486
year_built         0.571849
year_remod/add     0.550370
full_bath          0.537969
totrms_abvgrd      0.504014
has_fireplace      0.485858
has_mas_vnr        0.406245
open_porch_sf      0.333476
wood_deck_sf       0.326490
lot_area           0.296566
half_bath          0.283001
2nd_flr_sf         0.248452
has_garage         0.230855
has_bsmt           0.157385
bedroom_abvgr      0.137067
screen_porch       0.134581
3ssn_porch         0.048732
mo_sold            0.032735
has_pool           0.024366
misc_val          -0.007375
yr_sold           -0.015203
low_qual_fin_sf   -0.041594
id                -0.051398
ms_subclass       -0.087335
overall_cond      -0.097019
kitchen_abvgr     -0.125444
enclosed_porch    -0.135656
has_fence         -0.181758
pid               -0.255052
Name: saleprice, dtype: float64

In [ ]:
# overall_qual       0.800207 /
# gr_liv_area        0.697038
# 1st_flr_sf         0.618486
# year_built         0.571849
# year_remod/add     0.550370
# full_bath          0.537969
# totrms_abvgrd      0.504014

# currently included
# 'lot_area', 'overall_qual', 'overall_cond'

# drop overall cond

In [20]:
y = df['saleprice']
features = ['lot_area', 'overall_qual', 'overall_cond',
            'gr_liv_area', '1st_flr_sf', 'year_built', 'year_remod/add', 'full_bath', 'totrms_abvgrd',
    'has_mas_vnr', 'has_bsmt', 'has_fireplace', 'has_garage', 'has_pool', 'has_fence'
    ]

X = df[features]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [22]:
lr = LinearRegression()
lr.fit(X_train_sc ,y_train)

print('train', lr.score(X_train_sc, y_train))
print('test', lr.score(X_test_sc, y_test))

train 0.782080229794
test 0.817199558198


In [39]:
enet = ElasticNet()
params = {
    'alpha': np.arange(.7, 1, .005),
    'l1_ratio': np.arange(.01, 1.0, .005)
}
gs = GridSearchCV(enet, params)
gs.fit(X_train_sc, y_train)

print(gs.best_params_)
gs.score(X_test_sc, y_test)

{'alpha': 0.99500000000000022, 'l1_ratio': 0.89499999999999991}


0.80905497201164711

In [ ]:
# {'alpha': 0.75000000000000011, 'l1_ratio': 0.8600000000000001}
# 0.80901386751125381

# {'alpha': 0.80000000000000004, 'l1_ratio': 0.86999999999999988}
# 0.80909277483215436

# {'alpha': 0.99500000000000022, 'l1_ratio': 0.89499999999999991}
# 0.80905497201164711

# trying just lasso and just ridge

In [44]:
from sklearn.linear_model import LassoCV, RidgeCV

In [46]:
l_alphas = np.arange(.001, .15, .0025)
r_alphas = np.logspace(0, 5, 200)

In [49]:
best_lasso = LassoCV(alphas=l_alphas)
best_lasso.fit(X_train_sc, y_train)
print('Lasso Alpha', best_lasso.alpha_)
print(best_lasso.score(X_train_sc, y_train))

best_ridge = RidgeCV(alphas=r_alphas)
best_ridge.fit(X_train_sc, y_train)
print('Ridge Alpha', best_ridge.alpha_)
print(best_lasso.score(X_train_sc, y_train))

Lasso Alpha 0.146
0.782080229742
Ridge Alpha 128.989026125
0.782080229742


# Back to Work

In [10]:
df['lot_shape'].value_counts()

Reg    1295
IR1     692
IR2      55
IR3       9
Name: lot_shape, dtype: int64

In [21]:
df.groupby(['lot_shape'])['saleprice'].mean()

lot_shape
IR1    211848.670520
IR2    233954.745455
IR3    193182.222222
Reg    162925.812355
Name: saleprice, dtype: float64

In [32]:
# if ir3 replace with ir2
df['lot_shape'] =df['lot_shape'].replace({'IR3':'IR2'})

In [33]:
df['lot_shape'].value_counts()

Reg    1295
IR1     692
IR2      64
Name: lot_shape, dtype: int64